Script Deployment: Deploy and score a machine learning model by using an online endpoint using Scoring Script

**Workflow** 

1. Initialize Workspace & create workspace handle 
2. Initialize Environment 
3. configure Endpoint 
4. Get reference of the register model 
5. create a scoring script 
6. Configure Deployment 
7. Assign traffic to the deployment 
8. Test the Deployment 
9. Delete the Deployment

**Step 1: Initialize Workspace and Create Workspace handle**

In [1]:
from azureml.core import Workspace
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential

# Initialize  workspace
ws = Workspace.from_config()  

# Get a handle to the workspace
credential = DefaultAzureCredential()  # authenticate
ml_client = MLClient( credential=credential,
                      subscription_id=ws.subscription_id,
                      resource_group_name=ws.resource_group,
                      workspace_name=ws.name,
                    )


**Step 2: Initialize Environment**

In [2]:
import os
from azure.ai.ml.entities import Environment

custom_env_name  = "ENV-SDKv2"
# dependencies_dir = '../dependencies'
# env = Environment( name=custom_env_name,
#                    description="Evironment for python SDKv2 Execution",
#                    conda_file=os.path.join(dependencies_dir, "conda.yaml"),
#                    image="mcr.microsoft.com/azureml/openmpi4.1.0-ubuntu20.04:latest",
#                  )
# env = ml_client.environments.create_or_update(env)

# GET ENVIRONMENT
# use 'label' parameter to get latest environment for example label='latest'
# use 'version' parameter to get specific version environment, for example version=2
env = ml_client.environments.get(name=custom_env_name, label='latest') 

print(f"Environment with name {env.name} is registered to workspace, the environment version is {env.version}")

Environment with name ENV-SDKv2 is registered to workspace, the environment version is 6


Step 3: Configure Endpoint

**Note:** auth_mode is either **key** or **token**. The key difference is **token** expires and **key** won't

In [14]:
from azure.ai.ml.entities import ManagedOnlineEndpoint

# online endpoint
online_endpoint_name = "pima-endpoint-SDKv2-4"

endpoint = ManagedOnlineEndpoint(
    name=online_endpoint_name,
    description="this is a sample online endpoint for mlflow model",
    auth_mode="key",
    tags={"SDK": "2"},
)
# create endpoint
ml_client.begin_create_or_update(endpoint).result()

ManagedOnlineEndpoint({'public_network_access': 'Enabled', 'provisioning_state': 'Succeeded', 'scoring_uri': 'https://pima-endpoint-sdkv2-4.centralindia.inference.ml.azure.com/score', 'openapi_uri': 'https://pima-endpoint-sdkv2-4.centralindia.inference.ml.azure.com/swagger.json', 'name': 'pima-endpoint-sdkv2-4', 'description': 'this is a sample online endpoint for mlflow model', 'tags': {'SDK': '2'}, 'properties': {'azureml.onlineendpointid': '/subscriptions/ba5d6a04-af22-45ea-bc5a-946ef1c32949/resourcegroups/us_azure_practice/providers/microsoft.machinelearningservices/workspaces/us_azure/onlineendpoints/pima-endpoint-sdkv2-4', 'AzureAsyncOperationUri': 'https://management.azure.com/subscriptions/ba5d6a04-af22-45ea-bc5a-946ef1c32949/providers/Microsoft.MachineLearningServices/locations/centralindia/mfeOperationsStatus/oe:a78ffdcd-763f-4353-b2d5-5456db1b024f:086f2100-fdb3-4c79-9ad9-12ffac9366cd?api-version=2022-02-01-preview'}, 'print_as_yaml': True, 'id': '/subscriptions/ba5d6a04-af22

**Step 4: Get referrence to Registered Model**

In [4]:
# referencing model from model registry
model_name = 'pima_model_SDKv2_02'
model = ml_client.models.get(name=model_name, label="latest")

**Step 5: Create a Scoring Script and save in 'src' folder**

**Step 6: Configure Deployment**

**Instance type**: The VM size to use for the deployment. For the list of supported sizes.[cick here](https://learn.microsoft.com/en-us/azure/machine-learning/reference-managed-online-endpoints-vm-sku-list?view=azureml-api-2)

In [16]:
from azure.ai.ml.entities import ManagedOnlineDeployment, CodeConfiguration

deployment_name   = "script-deployment"
deployment_script = ManagedOnlineDeployment(
                name=deployment_name, # deployment name
                endpoint_name=online_endpoint_name,  
                model=model,
                environment=env,
                code_configuration=CodeConfiguration(
                code="../src", scoring_script="pima_scoreOnlineEndpoint_SDKv2.py"
                ),
                instance_type="Standard_F4s_v2",
                instance_count=1,
            )

ml_client.online_deployments.begin_create_or_update(deployment=deployment_script).result() # create local deployment


Check: endpoint pima-endpoint-SDKv2-4 exists


......................................................................

ManagedOnlineDeployment({'private_network_connection': None, 'provisioning_state': 'Succeeded', 'endpoint_name': 'pima-endpoint-sdkv2-4', 'type': 'Managed', 'name': 'script-deployment', 'description': None, 'tags': {}, 'properties': {'AzureAsyncOperationUri': 'https://management.azure.com/subscriptions/ba5d6a04-af22-45ea-bc5a-946ef1c32949/providers/Microsoft.MachineLearningServices/locations/centralindia/mfeOperationsStatus/od:a78ffdcd-763f-4353-b2d5-5456db1b024f:17eddca0-a39d-494b-af55-3c93a5db18b5?api-version=2023-04-01-preview'}, 'print_as_yaml': True, 'id': '/subscriptions/ba5d6a04-af22-45ea-bc5a-946ef1c32949/resourceGroups/us_azure_practice/providers/Microsoft.MachineLearningServices/workspaces/us_azure/onlineEndpoints/pima-endpoint-sdkv2-4/deployments/script-deployment', 'Resource__source_path': None, 'base_path': '/mnt/batch/tasks/shared/LS_root/mounts/clusters/instancenotebook/code/Users/uj_az/PIMA- OnlineEndpoint/02-Deploying-Online-Endpoint/SDKv2/notebooks', 'creation_context

**Step 7: Assign traffic to the deployment**

In [17]:
# assign traffic to the deployment
endpoint.traffic = {deployment_name: 100}
ml_client.begin_create_or_update(endpoint).result()

ManagedOnlineEndpoint({'public_network_access': 'Enabled', 'provisioning_state': 'Succeeded', 'scoring_uri': 'https://pima-endpoint-sdkv2-4.centralindia.inference.ml.azure.com/score', 'openapi_uri': 'https://pima-endpoint-sdkv2-4.centralindia.inference.ml.azure.com/swagger.json', 'name': 'pima-endpoint-sdkv2-4', 'description': 'this is a sample online endpoint for mlflow model', 'tags': {'SDK': '2'}, 'properties': {'azureml.onlineendpointid': '/subscriptions/ba5d6a04-af22-45ea-bc5a-946ef1c32949/resourcegroups/us_azure_practice/providers/microsoft.machinelearningservices/workspaces/us_azure/onlineendpoints/pima-endpoint-sdkv2-4', 'AzureAsyncOperationUri': 'https://management.azure.com/subscriptions/ba5d6a04-af22-45ea-bc5a-946ef1c32949/providers/Microsoft.MachineLearningServices/locations/centralindia/mfeOperationsStatus/oe:a78ffdcd-763f-4353-b2d5-5456db1b024f:1abc1d85-7c07-4c7c-9c46-2ccf4d4f0251?api-version=2022-02-01-preview'}, 'print_as_yaml': True, 'id': '/subscriptions/ba5d6a04-af22

**Step 8: Test the endpoint with sample data**

We will send a sample request using a [json](../../data/sample_request.json) file.


In [9]:
#import json
#
#json.loads("../../data/sample_request.json")

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [18]:
# testing the endpoint
ml_client.online_endpoints.invoke(
    endpoint_name=online_endpoint_name,
    deployment_name=deployment_name,
    request_file="../../data/sample_request.json",
)

'[0, 0]'